In [62]:
import re
import math
import pickle
import string
from collections import Counter
import matplotlib.pyplot as plt
import requests
from download_text import get_grams_from_text
from corus.sources.meta import METAS
from corus.readme import format_metas, show_html, patch_readme
from corus import load_lenta

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
w = pickle.load(open("../../../word_data/words.pkl", "rb"))

In [13]:
def tokens(text, language):
    shab = r'[a-z]+' if language == 'eng' else r'[а-я]+'
    """Возвращает список токенов (подряд идущих буквенных последовательностей) в тексте. 
       Текст при этом приводится к нижнему регистру."""
    return re.findall(shab, text.lower())

def pdist(counter):
    "Превращает частоты из Counter в вероятностное распределение."
    N = sum(list(counter.values()))
    return lambda x: counter[x] / N

def Pwords(words):
    "Вероятности слов, при условии, что они независимы."
    return product(P(w) for w in words)

def product(nums):
    "Перемножим числа.  (Это как `sum`, только с умножением.)"
    result = 1
    for x in nums:
        result *= x
    return result

P = pdist(COUNTS)

WORDS = w[1]
ALPHABET = {'ENG': 'abcdefghijklmnopqrstuvwxyz',
            'RUS': 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'}
COUNTS = WORDS

## 4.2 Вероятностные подходы - разбение слова на сегменты

**Задача**: *Разбить полученную последовательность символов без пробелов на последовательность слов

Вероятностные подходы основаны на оценке встречаемости каждого слова корпуса. В данном случае выбирается наиболее вероятная подпоследовательность слов, то есть:

$$s^* = \underset{s \in S}{\operatorname{s \in S}} P(s) \qquad (1)$$

где $S$ - множество всех всевозможных подпоследовательностей слов из данной строки.

### 4.2.1 Вероятностная языковая модель

Наилучшая последовательность слов определяется формулой (1). Тогда при $n$ словах в последовательности $s$ вероятность разбиения:

$$P(s) = \prod\limits_{k=1}^n P(w_k)$$

где $P(w_k)$ - вероятность встретить в тексте слово $w_k$.

Рассмотрим строку `the men dine here`. Для оценки вероятностей воспользуемся открытым [словарем](http://norvig.com/ngrams/). Данный словарь содержит более триллиона слов. В используемом корпусе приведенные слова встретились:

- `the` - 23135851162 раза;
- `men` - 174058407 раз;
- `dine` - 2012279 раз;
- `here` – 639711198 раз

Всего слов в корпусе: `1024908267229`. Тогда:

$$P(\operatorname{the men dine here}) = P(\operatorname{the}) P(\operatorname{men}) P(\operatorname{die}) P(\operatorname{here}) = 4.698 × 10-1$$

Так как $P(w) \in [0,1]$, то при перемножении вероятностей возникает проблема арифметического переполнения снизу. Чтобы этого избежать часто прибегают к логарифмированию:

$$\log P(s) = \log \prod\limits_{k=1}^n P(w_k) = \sum\limits_{k=1}^n \log P(w_k)$$

Другой возможной проблемой является **отсутствие встретившегося слова** в словаре. Оценка вероятности встретить данное слово равно нулю. Чтобы не возникало такой ситуации, применяют **аддитивное сглаживание (сглаживание Лапласа)**, которое состоит в
искусственном *добавлении единицы к встречаемости каждого слова*:

$$P(w_i) = \frac{v_i + 1}{\sum\limits_{i \in V} (v_i+1)} = \frac{v_i+1}{|V| + \sum\limits_{i \in V}v_i}$$

где $v_i$ - сколько раз встретилось слово $w_i$, $V$ - словарь.

Соответственно, **1-ый подход** заключается в следующем: перенумеруем все возможные разбиения и выберем то, у которого максимальная `Pwords`

Как выбрать количество сегментов для строки длины *n* - этим мы занимались в прошлых пунтах.

### 4.2.2. `First Word + Other Words`

**Подход 2:** Делаем одно разбиение - на первое слово и все остальное.  Если предположить, что слова независимы,
можно максимизировать вероятность первого слова + лучшего разбиения оставшихся букв.
    
    assert segment('choosespain') == ['choose', 'spain']

    segment('choosespain') ==
       max(Pwords(['c'] + segment('hoosespain')),
           Pwords(['ch'] + segment('oosespain')),
           Pwords(['cho'] + segment('osespain')),
           Pwords(['choo'] + segment('sespain')),
           ...
           Pwords(['choosespain'] + segment('')))
       
    
       
Чтобы сделать это хоть сколько-нибудь эффективным, нужно избежать слишком большого числа пересчетов оставшейся части слова.  Это можно сделать или с помощью [динамического программирования](https://habr.com/ru/post/113108/) или с помощью [мемоизации](https://ru.wikipedia.org/wiki/Мемоизация) кэширования. 

Кроме того, для первого слова не обязательно брать все возможные варианты разбиений - мы можем установить максимальную длину - чуть большую, чем длина самого длинного слова, которое мы видели.

In [41]:
def memo(f):
    "Запомнить результаты исполнения функции f, чьи аргументы args должны быть хешируемыми."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

- [мемоизация в python](https://habr.com/ru/post/335866/)

In [20]:
new = {len(w): w for w in COUNTS}

In [34]:
def splits(text, start=0, L=29):
    "Вернуть список всех пар (a, b); start <= len(a) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

In [39]:
print(splits('слово'))
print(splits('оченьдлинноеслово', 1, 10))

[('', 'слово'), ('с', 'лово'), ('сл', 'ово'), ('сло', 'во'), ('слов', 'о'), ('слово', '')]
[('о', 'ченьдлинноеслово'), ('оч', 'еньдлинноеслово'), ('оче', 'ньдлинноеслово'), ('очен', 'ьдлинноеслово'), ('очень', 'длинноеслово'), ('оченьд', 'линноеслово'), ('оченьдл', 'инноеслово'), ('оченьдли', 'нноеслово'), ('оченьдлин', 'ноеслово'), ('оченьдлинн', 'оеслово')]


In [42]:
@memo
def segment(text):
    "Вернуть список слов, который является наиболее вероятной сегментацией нашего текста."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [43]:
segment('новаястрана')

['новая', 'страна']

In [44]:
segment('бонуснаяигра')

['бонусная', 'игра']

In [56]:
print(new[54])
print(segment(new[54]))
print(Pwords(segment(new[54])))
print(Pwords(segment(new[54] * 2)))
print(Pwords(segment(new[54] * 7))) #проблема переполнения разрядности чисел

тыполюбитьзаставиласебячтобыплеснутьмневдушучернымядом
['ты', 'полюбить', 'заставила', 'себя', 'чтобы', 'плеснуть', 'мне', 'в', 'душу', 'черным', 'ядом']
2.0531571169041637e-48
4.215454146694218e-96
0.0


In [49]:
print(new[31])
print(segment(new[31]))

чемпионатфотографийплохойсобаки
['чемпионат', 'фотографий', 'плохой', 'собаки']


- выглядит неплохо
- предположение о мешке слов имеет ряд ограничений.
- пересчет Pwords на каждом вызове выглядит неэффективным.
- переполнение чисел возникает для текстов длинее +- 100 слов; придется использовать логарифмическое сглаживание


## Динамическое программирование. Насколько дорого превращать одно слово в другое?

<a name='4-1'></a>

Динамическое программирование позволяет разбить задачу на подзадачи, решив которые можно скомпоновать финальное решение. Будем пытаться превратить строку *source* `[0..i]` в строку *target* `[0..j]`, сосчитаем все возможные комбинации подстрок `substrings[i, j]` и рассчитаем их `edit_distance` до исходной строки. Будем сохранять результаты в таблицу и переиспользовать их для расчета дальнейших изменений.

Необходимо создать матрицу такого вида:  

$$\text{Initial}$$

\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \tag{4}\\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}

$$\text{Operation}$$
\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}

Таким образом, превратить слово **play** в слово **stay** при стоимости вставки 1, стоимости удаления 1, и стоимости замены 2 даст такую таблицу:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>




In [57]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    '''
    Input: 
        source: строка-исходник
        target: строка, в которую мы должны исходник превратить
        ins_cost: цена вставки
        del_cost: цена удаления
        rep_cost: цена замены буквы
    Output:
        D: матрица размера len(source)+1 на len(target)+1 содержащая минимальные расстояния edit_distance
        med: минимальное расстояние edit_distance (med), необходимое, 
        чтобы превратить строку source в строку target
    '''
    # стоимость удаления и вставки = 1
    m = len(source)
    n = len(target)

    # Заткнем нашу матрицу нулями
    D = np.zeros((m + 1, 
                  n + 1), dtype=int) 
    
    # Заполним первую колонку
    for row in range(1, m + 1): 
        D[row, 0] = D[row - 1, 0] + del_cost
        
    # Заполним первую строку
    for col in range(1, n+1): 
        D[0, col] = D[0, col - 1] + ins_cost
        
    # Теперь пойдем от 1 к m-той строке
    for row in range(1, m + 1): 
        
        # итерируемся по колонкам от 1 до n
        for col in range(1, n + 1):
            
            # r_cost - стоимость замены
            r_cost = rep_cost
            
            # Совпадает ли буква исходного слова из предыдущей строки
            # с буквой целевого слова из предыдущей колонки, 
            if source[row - 1] == target[col - 1]:
                # Если они не нужны, то замена не нужна -> стоимость = 0
                r_cost = 0
                
            # Обновляем значение ячейки на базе предыдущих значений 
            # Считаем D[i,j] как минимум из трех возможных стоимостей (как в формуле выше)
            D[row, col] = min([D[row - 1, col] + del_cost, 
                               D[row, col - 1] + ins_cost, 
                               D[row - 1, col - 1] + r_cost])
          
    # установить edit_distance в значение из правого нижнего угла
    med = D[m,n]
    

    return D, med

In [60]:
import numpy as np
import pandas as pd

source =  'playds'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)

print("Расстояние: ",min_edits, "\n")

idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

Расстояние:  6 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4
d  5  6  7  6  5
s  6  5  6  7  6


### 4.2.2 Биграммы

Предудыщий метод рассматривает каждое слово в отдельности, однако не учитывает соседние слова, которые также могут помочь правильно разбить текст. Чтобы учесть сочетания слов рассматриваются **биграммы** – сочетания по два слова и на основе вероятностей встречаемости *сочетаний слов* восстановить пробелы.

Строка `the men dine here` с учётом начала и конца предложения может быть разбита на $5$ биграмм:

1. begin the
2. the men
3. men dine
4. dine here
5. here end

Формула (2) тогда принимает вид:

$$P(s) = \prod\limits_{k=1}^n P(w_k | w_{k-1})$$

Чуть менее неправильная аппроксимация:
    
$P(w_1 \ldots w_n) = P(w_1) \times P(w_2 \mid w_1) \times P(w_3 \mid w_2) \ldots  \times \ldots P(w_n \mid w_{n-1})$

Эта штука называется *биграммной* моделью. Представьте, что вы взяли текст, достали из него все возможные пары подряд идущих слов и положили каждую пару в мешок, промаркированный ПЕРВЫМ словом из пары.  После этого, чтобы сгенерировать кусок текста, мы берем первое слово из исходного мешка слов , а каждое следующее слово вынимаем из соответствующего мешка биграмм. 

Начнем с определения вероятности текущего слова при условии данного предыдущего слова из Counter:

Отмечу, что для английского языка биграммная модель будет выглядеть так:
    
$P(w_1 \ldots w_n) = P(w_1) \times P(w_2 \mid w_1) \times P(w_3 \mid w_2) \ldots  \times \ldots P(w_n \mid w_{n-1})$

условная вероятность слова при условии предыдущего слова определяется так:

$P(w_n \mid w_{n-1}) = P(w_{n-1}w_n) / P(w_{n-1}) $

In [65]:
bigrams = get_grams_from_text(path='../../../word_data/lenta-ru-news.csv.gz',
                              n=[2], 
                              amount_of_sentense=15000, 
                              show_how_much=5000, 
                              delete_stop_words=False)

Sentence 5000
Sentence 10000


In [69]:
COUNTS1 = w[1]
COUNTS2 = bigrams[2]

In [71]:
print(len(COUNTS1), sum(list(COUNTS1.values())))
print(len(COUNTS2), sum(list(COUNTS2.values())))

337838 16167199
1062637 2401223


In [76]:
P1w = pdist(COUNTS1)
P2w = pdist(COUNTS2)

In [79]:
def Pwords2(words, prev='<S>'):
    "Вероятность последовательности слов с помощью биграммной модели(при условии предыдущего слова)."
    return product(cPword(w, (prev if (i == 0) else words[i-1]) )
                   for (i, w) in enumerate(words))

# Перепишем Pwords на большой словарь P1w вместо Pword
def Pwords(words):
    "Вероятности слов при условии их независимости."
    return product(P1w(w) for w in words)

def cPword(word, prev):
    "Условная вероятность слова при условии предыдущего."
    bigram = prev + ' ' + word
    if P2w(bigram) > 0 and P1w(prev) > 0:
        return P2w(bigram) / P1w(prev)
    else: # если что-то не встретилось, поставим среднее между P1w и 0
        return P1w(word) / 2

In [82]:
print(Pwords(tokens('я знаю эту книгу', language='rus')))
print(Pwords2(tokens('я знаю эту книгу', language='rus')))
print(Pwords2(tokens('эту книгу я знаю', language='rus')))
print(Pwords2(tokens('эту книгу знаю я', language='rus')))

6.4317520743662085e-16
2.2642865047148052e-14
2.2642865047148052e-14
4.0198450464788803e-17


Чтобы сделать `segment2`, скопируем `segment`, добавим в аргументы предыдущий токен, а вероятности будем считать с помощью `Pwords2` вместо `Pwords`.

In [83]:
@memo 
def segment2(text, prev='<S>'): 
    "Возвращает наилучшее разбиение текста, используя статистику биграмм." 
    if not text: 
        return []
    else:
        candidates = ([first] + segment2(rest, first) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=lambda words: Pwords2(words, prev))

In [88]:
print(segment2('новаястрана'))
print(segment2('большаяималенькаястрана'))
print(segment2('произведениеискусстваневообразимойкрасоты'))
print(segment2(new[54]))

['новая', 'страна']
['большая', 'и', 'маленькая', 'страна']
['произведение', 'искусства', 'невообразимой', 'красоты']
['ты', 'полюбить', 'заставила', 'себя', 'чтобы', 'плеснуть', 'мне', 'в', 'душу', 'черным', 'ядом']


In [91]:
oxxy = """Там, где нас нет, горит невиданный рассвет.
Где нас нет - море и рубиновый закат.
Где нас нет - лес, как малахитовый браслет.
Где нас нет, на Лебединых островах.
Где нас нет, услышь меня и вытащи из омута.
Веди в мой вымышленный город, вымощенный золотом.
Во сне я вижу дали иноземные.
Где милосердие правит, где берега кисельные."""
oxxy = ''.join(re.findall(r'[а-я]',oxxy.lower()))
oxxy

'тамгденаснетгоритневиданныйрассветгденаснетмореирубиновыйзакатгденаснетлескакмалахитовыйбраслетгденаснетналебединыхостровахгденаснетуслышьменяивытащиизомутаведивмойвымышленныйгородвымощенныйзолотомвоснеявижудалииноземныегдемилосердиеправитгдеберегакисельные'

In [92]:
print(segment(oxxy))
print(segment2(oxxy))

['там', 'где', 'нас', 'нет', 'горит', 'не', 'в', 'и', 'данный', 'рассвет', 'где', 'нас', 'нет', 'море', 'и', 'рубиновый', 'закат', 'где', 'нас', 'нет', 'лес', 'как', 'малахит', 'о', 'вый', 'браслет', 'где', 'нас', 'нет', 'на', 'лебеди', 'ных', 'островах', 'где', 'нас', 'нет', 'услышь', 'меня', 'и', 'вы', 'та', 'щи', 'из', 'омут', 'а', 'веди', 'в', 'мой', 'вымышленный', 'город', 'вы', 'мощен', 'ный', 'золотом', 'во', 'с', 'не', 'я', 'вижу', 'дали', 'иноземные', 'где', 'милосердие', 'правит', 'где', 'берега', 'кисельные']
['там', 'где', 'нас', 'нет', 'горит', 'не', 'в', 'и', 'данный', 'рассвет', 'где', 'нас', 'нет', 'море', 'и', 'рубиновый', 'закат', 'где', 'нас', 'нет', 'лес', 'как', 'малахит', 'о', 'вый', 'браслет', 'где', 'нас', 'нет', 'на', 'лебеди', 'ных', 'островах', 'где', 'нас', 'нет', 'услышь', 'меня', 'и', 'вы', 'та', 'щи', 'из', 'омут', 'а', 'веди', 'в', 'мой', 'вымышленный', 'город', 'вы', 'мощен', 'ный', 'золотом', 'во', 'сне', 'я', 'вижу', 'дали', 'иноземные', 'где', 'милос

In [93]:
pickle.dump(bigrams, open("../../../word_data/bigrams.pkl", "wb"))

In [94]:
bg = pickle.load(open("../../../word_data/bigrams.pkl", "rb"))

## 4.2.3 Осталась валидация - насколько модели качественные